# Vorbereitung

In [227]:
# !pip install mysql
# !pip install mysql-connector-python
# !pip install pymysql
# !pip install SQLAlchemy

In [228]:
from sqlalchemy import create_engine
import mysql.connector
from mysql.connector import errorcode
from hilfsfunktionen import *
import pandas as pd
from IPython.display import display_html
import pymysql

In [229]:
mysql_usr = "BIA_User" # mit eurem Nutzer ersetzen
mysql_pw = "BIA_User_Passwort" # mit eurem Passwort ersetzen

# Extract

In [230]:
connection = create_server_connection("localhost", mysql_usr, mysql_pw)

create_database_query = "DROP DATABASE IF EXISTS OLTP_AirBnB_db;"
create_database(connection, create_database_query)

create_database_query = "CREATE DATABASE OLTP_AirBnB_db;"
create_database(connection, create_database_query)

db_connection_str = 'mysql+pymysql://' + mysql_usr + ':' + mysql_pw + '@localhost/OLTP_AirBnB_db'
db_connection = create_engine(db_connection_str)

MySQL Database connection successful
Database created successfully
Database created successfully


In [231]:
db_connection_str = 'mysql+pymysql://' + mysql_usr + ':' + mysql_pw + '@localhost/OLTP_AirBnB_db'
db_connection = create_engine(db_connection_str)

In [232]:
connection = create_db_connection("localhost", mysql_usr, mysql_pw, "OLTP_AirBnB_db")
execute_query(connection, "Show Tables") 

MySQL Database connection successful
result: []
Query successful


### Mietpreise Berlin

In [233]:
df_mietpreise = pd.read_csv("../data/mietpreise_berlin.csv", sep=";", index_col=0, dtype={'MietpreisID': 'int', 'Ortsteil': 'string', 'Preis_Pro_m2': 'float'}, encoding= 'unicode_escape')

In [234]:
df_mietpreise.head()

,Ortsteil,Preis_Pro_m2
MietpreisID,,
1,Adlershof,26.84
2,Alt-Hohenschönhausen,14.01
3,Altglienicke,17.39
4,Baumschulenweg,15.37
5,Biesdorf,21.10


In [235]:
df_mietpreise.to_sql(con=db_connection, name='rents', if_exists='replace')

74

### Sehenswürdigkeiten Berlin

In [236]:
df_sehenswuerdigkeiten = pd.read_csv("../data/sehenswuerdigkeiten_berlin.csv", sep=";", index_col=0, dtype={'Sehenswürdigkeiten': 'string', 'Ort': 'string', 'Art': 'string', 'Besucher pro Jahr': 'int64', 'Longitude': 'float', 'Latitude': 'float'}, encoding= 'unicode_escape')
df_sehenswuerdigkeiten['ID'] = np.arange(len(df_sehenswuerdigkeiten)) + 1
df_sehenswuerdigkeiten.head()
df_sehenswuerdigkeiten = df_sehenswuerdigkeiten.reset_index().set_index('ID', inplace=False)
# print(df_sehenswuerdigkeiten)

In [237]:
df_sehenswuerdigkeiten.head()

,Sehenswürdigkeiten,Ort,Art,Besucher pro Jahr,Longitude,Latitude
ID,,,,,,
1,Reichstagsgebäude,Mitte,Wahrzeichen & Sehenswürdigkeit  Architektonis...,2000000,13376198.0,52518623.0
2,Brandenburger Tor,Mitte,Wahrzeichen & Sehenswürdigkeiten  Monumente &...,3000000,13377775.0,52516266.0
3,Holocaust-Mahnmal,Mitte,Monumente & Statuen,480000,13378344.0,52513923.0
4,Topographie des Terrors,Friedrichshain-Kreuzberg,Spezialmuseen,1000000,133763318.0,525041646.0
5,Gedenkstätte Berliner Mauer,Mitte,Historische Stätten,750000,1339019.0,52535052.0


In [238]:
df_sehenswuerdigkeiten.to_sql(con=db_connection, name='sights', if_exists='replace')

37

### ÖPNV

In [239]:
df_busstationen = pd.read_csv("../data/stops.csv", sep=",", index_col=0, encoding= 'unicode_escape')
df_busstationen['ID'] = np.arange(len(df_busstationen)) + 1
df_busstationen.reset_index()
df_busstationen.set_index('ID', inplace=True)

In [240]:
df_busstationen.head()

,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code,zone_id,level_id
ID,,,,,,,,,,,
1,NaN,S Oranienburger Str. (Berlin),Ersatzhalt TucholskystraÃe vor Oranienburger ...,52.524724,13.392833,0,NaN,0,NaN,5555 S Oranienburger Str. (Berlin),4.0
2,NaN,S Blankenfelde (TF) Bhf,S-Bahnsteig Gleis 3,52.338503,13.415406,0,de:12072:900245027,1,3,5955 S Blankenfelde (TF) Bhf,33.0
3,NaN,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 2,52.960114,11.949402,0,de:12070:900215110,1,2,"4533 Bad Wilsnack, Bahnhof",50.0
4,NaN,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 3,52.960219,11.949528,0,de:12070:900215110,1,3,"4533 Bad Wilsnack, Bahnhof",50.0
5,NaN,"PrÃ¶sen, Bahnhof",Bahnsteig Gleis 1,51.434919,13.488216,0,de:12062:900415465,1,1,"7958 PrÃ¶sen, Bahnhof",4.0


In [241]:
# for index, row in enumerate(df_busstationen["stop_name"]):
#     if 'Ã¼' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¤', 'ä')

In [242]:
# df_busstationen = df_busstationen

# for index, row in enumerate(df_busstationen["stop_desc"]):    
#     if 'Ã¼' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¤', 'ä')

In [243]:
# df_busstationen = df_busstationen

# for index, row in enumerate(df_busstationen["stop_desc"]):    
#     if 'Ã¼' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¤', 'ä')

In [244]:
# for index in range(len(df_busstationen)):
#     if 'Ã¼' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["stop_name"][index + 1]:
#         df_busstationen["stop_name"][index + 1] = df_busstationen["stop_name"][index + 1].replace('Ã¤', 'ä')

#     if 'Ã¼' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["stop_desc"][index + 1]:
#         df_busstationen["stop_desc"][index + 1] = df_busstationen["stop_desc"][index + 1].replace('Ã¤', 'ä')

#     if 'Ã¼' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¼', 'ü')
#     if 'Ã' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã', 'ß')
#     if 'Ã¶' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¶', 'ö')
#     if 'Ã¤' in df_busstationen["zone_id"][index + 1]:
#         df_busstationen["zone_id"][index + 1] = df_busstationen["zone_id"][index + 1].replace('Ã¤', 'ä')


In [245]:
df_busstationen.to_sql(con=db_connection, name='stops', if_exists='replace')

41078

### Straftaten in Berlin

In [246]:
df_berlin_crimes = pd.read_csv("../data/Berlin_crimes.csv", sep=",", index_col=0, encoding= 'unicode_escape')
df_berlin_crimes['ID'] = np.arange(len(df_berlin_crimes)) + 1
df_berlin_crimes = df_berlin_crimes.reset_index().set_index('ID', inplace=False)

In [247]:
df_berlin_crimes.head()

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local
ID,,,,,,,,,,,,,,,,,,,,
1,2012,Mitte,10111,Tiergarten SÃ¼d,70,46,586,194,118,2263,18,328,120,68,16,4,273,26,171,1032
2,2012,Mitte,10112,Regierungsviertel,65,29,474,123,142,3203,10,307,170,37,10,4,380,124,98,870
3,2012,Mitte,10113,Alexanderplatz,242,136,1541,454,304,8988,81,792,822,275,49,27,1538,522,435,3108
4,2012,Mitte,10114,BrunnenstraÃe SÃ¼d,52,25,254,60,66,1916,86,192,396,131,14,5,428,122,213,752
5,2012,Mitte,10221,Moabit West,130,51,629,185,199,2470,94,410,325,161,42,22,516,64,259,1403


In [248]:
for index in range(len(df_berlin_crimes)):
    if 'Ã¼' in df_berlin_crimes["Location"][index + 1]:
        df_berlin_crimes["Location"][index + 1] = df_berlin_crimes["Location"][index + 1].replace('Ã¼', 'ü')
    if 'Ã' in df_berlin_crimes["Location"][index + 1]:
        df_berlin_crimes["Location"][index + 1] = df_berlin_crimes["Location"][index + 1].replace('Ã', 'ß')
    if 'Ã¶' in df_berlin_crimes["Location"][index + 1]:
        df_berlin_crimes["Location"][index + 1] = df_berlin_crimes["Location"][index + 1].replace('Ã¶', 'ö')
    if 'Ã¤' in df_berlin_crimes["Location"][index + 1]:
        df_berlin_crimes["Location"][index + 1] = df_berlin_crimes["Location"][index + 1].replace('Ã¤', 'ä')

for index in range(len(df_berlin_crimes)):
    if 'Ã¼' in df_berlin_crimes["District"][index + 1]:
        df_berlin_crimes["District"][index + 1] = df_berlin_crimes["District"][index + 1].replace('Ã¼', 'ü')
    if 'Ã' in df_berlin_crimes["District"][index + 1]:
        df_berlin_crimes["District"][index + 1] = df_berlin_crimes["District"][index + 1].replace('Ã', 'ß')
    if 'Ã¶' in df_berlin_crimes["District"][index + 1]:
        df_berlin_crimes["District"][index + 1] = df_berlin_crimes["District"][index + 1].replace('Ã¶', 'ö')
    if 'Ã¤' in df_berlin_crimes["District"][index + 1]:
        df_berlin_crimes["District"][index + 1] = df_berlin_crimes["District"][index + 1].replace('Ã¤', 'ä')

In [249]:
for index in range(len(df_berlin_crimes)):
    if 'nicht zuzuordnen' in df_berlin_crimes["Location"][index + 1]:
        df_berlin_crimes["Location"][index + 1] = np.nan

In [250]:
df_berlin_crimes.head()

,Year,District,Code,Location,Robbery,Street_robbery,Injury,Agg_assault,Threat,Theft,Car,From_car,Bike,Burglary,Fire,Arson,Damage,Graffiti,Drugs,Local
ID,,,,,,,,,,,,,,,,,,,,
1,2012,Mitte,10111,Tiergarten Süd,70,46,586,194,118,2263,18,328,120,68,16,4,273,26,171,1032
2,2012,Mitte,10112,Regierungsviertel,65,29,474,123,142,3203,10,307,170,37,10,4,380,124,98,870
3,2012,Mitte,10113,Alexanderplatz,242,136,1541,454,304,8988,81,792,822,275,49,27,1538,522,435,3108
4,2012,Mitte,10114,Brunnenstraße Süd,52,25,254,60,66,1916,86,192,396,131,14,5,428,122,213,752
5,2012,Mitte,10221,Moabit West,130,51,629,185,199,2470,94,410,325,161,42,22,516,64,259,1403


In [251]:
df_berlin_crimes.to_sql(con=db_connection, name='crimes', if_exists='replace')

1200

### AirBnBs in Berlin

In [252]:
df_berlin_train_airbnb = pd.read_csv("../data/train_airbnb_berlin.csv", sep=",", index_col=0, encoding= 'unicode_escape')

In [253]:
df_berlin_train_airbnb.head()

,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
Listing ID,,,,,,,,,,,,,,,,,,,,,
19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0
6436842.0,*,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,Berlin,...,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0
10559468.0,*,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0
27215482.0,*,193452785.0,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0
27287546.0,*,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,Berlin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,55.0


In [254]:
df_berlin_test_airbnb = pd.read_csv("../data/test_airbnb_berlin.csv", sep=",", index_col=0, encoding= 'unicode_escape')

In [255]:
df_berlin_test_airbnb.head()

,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,...,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready
Listing ID,,,,,,,,,,,,,,,,,,,,,
13338017.0,*,19153032.0,Sonfron,15-06-2014,within a few hours,91%,t,Kreuzberg,Friedrichshain-Kreuzberg,Berlin,...,05-05-2019,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f
2899239.0,*,1629859.0,Welgin,11-02-2011,within an hour,100%,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,27-02-2019,75.0,8.0,5.0,8.0,8.0,9.0,7.0,f,f
441586.0,*,153205.0,Leenamdr,12-05-2010,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,24-06-2017,94.0,10.0,9.0,10.0,9.0,9.0,9.0,t,f
19036720.0,*,129571502.0,Bertra&ÃGisn,07-03-2017,NaN,NaN,f,Tempelhof,Tempelhof - SchÃÂ¶neberg,Berlin,...,04-11-2018,96.0,10.0,10.0,10.0,10.0,9.0,9.0,t,f
17890052.0,*,28992128.0,Soscus,29-04-2015,within a day,67%,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,30-03-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f


In [256]:
df_berlin_airbnb = pd.concat([df_berlin_train_airbnb, df_berlin_test_airbnb], ignore_index=True)

In [257]:
df_berlin_airbnb.head()

,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0
1,*,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,Berlin,...,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0
2,*,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0
3,*,193452785.0,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0
4,*,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,Berlin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,55.0


In [258]:
len(df_berlin_airbnb)

23534

#### Fehlende Datenpunkte in der Spalte "Square Feet" Füllen

##### Decision Tree Regression-Algorithmus von sklearn trainieren

In [259]:
import numpy as np 
  
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [260]:
df_dataset = df_berlin_airbnb[["Postal Code", "Property Type", "Room Type", "Accomodates", "Bathrooms", "Bedrooms", "Beds", "Square Feet"]]
df_dataset.fillna(0, inplace=True)
df_dataset.loc[df_dataset["Square Feet"] == 0, "Square Feet"] = np.nan
df_dataset.loc[df_dataset["Accomodates"] == '*', "Accomodates"] = 0
df_dataset.loc[df_dataset["Postal Code"].isnull(), "Postal Code"] = 0
df_dataset.loc[df_dataset["Postal Code"] == '*', "Postal Code"] = 0
df_dataset.loc[df_dataset["Bathrooms"] == '*', "Bathrooms"] = 0
df_dataset.loc[df_dataset["Bedrooms"] == '*', "Bedrooms"] = 0
df_dataset.loc[df_dataset["Beds"] == '*', "Beds"] = 0
df_dataset = df_dataset[df_dataset["Square Feet"].notnull()]
df_dataset["Postal Code"].isnull().values.any()

False

In [261]:
df_property_type = df_dataset["Property Type"].unique()
df_property_type

array(['Apartment', 'Loft', 'Condominium', 'Guesthouse', 'House', '*',
       'Serviced apartment', 'Other', 'Guest suite'], dtype=object)

In [262]:

for index, p_type in enumerate(df_property_type):
    df_dataset.loc[df_dataset["Property Type"] == p_type, "Property Type"] = index

In [263]:
df_room_type = df_dataset["Room Type"].unique()
df_room_type

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [264]:

for index, r_type in enumerate(df_room_type):
    df_dataset.loc[df_dataset["Room Type"] == r_type, "Room Type"] = index

In [265]:
np.nan_to_num(df_dataset, copy=False)
# df_dataset.to_csv("../data/df_dataset.csv", sep=";", decimal=",")

array([['10961.0', 0, 0, ..., '2.0', '2.0', 831.0],
       ['10439.0', 0, 0, ..., '1.0', '1.0', 53.0],
       ['10829.0', 0, 0, ..., '1.0', '1.0', 235.0],
       ...,
       ['10317', 4, 0, ..., '4', '9', 2860.0],
       ['10119', 8, 0, ..., '1', '2', 344.0],
       ['10245', 0, 0, ..., '1', '2', 807.0]], dtype=object)

In [266]:
dataset = np.array(df_dataset)
np.nan_to_num(dataset, copy=False)

array([['10961.0', 0, 0, ..., '2.0', '2.0', 831.0],
       ['10439.0', 0, 0, ..., '1.0', '1.0', 53.0],
       ['10829.0', 0, 0, ..., '1.0', '1.0', 235.0],
       ...,
       ['10317', 4, 0, ..., '4', '9', 2860.0],
       ['10119', 8, 0, ..., '1', '2', 344.0],
       ['10245', 0, 0, ..., '1', '2', 807.0]], dtype=object)

In [267]:
x_features = dataset[:, :-1]
y_features = dataset[:, -1:]

In [268]:
x_train = x_features[:250]
y_train = y_features[:250]

x_test = x_features[250:]
y_test = y_features[250:]

In [269]:
# regressor = LinearRegression() 
# regressor.fit(x_features, y_features)
# score = regressor.score(x_train, y_train)
# print("R-squared:", score) 

In [270]:
# y_pred = regressor.predict(x_test)

# mse = mean_squared_error(y_test, y_pred)
# print("MSE: ", mse)
# print("RMSE: ", mse**(1/2.0)) 

In [271]:
# y_pred = regressor.predict([[10439, 1, 0, 1, 5, 4, 5]])
  
# # print the predicted price
# print("Predicted square feet: % d\n"% y_pred) 

In [272]:
regressor = DecisionTreeRegressor(random_state = 0) 
regressor.fit(x_features, y_features)

DecisionTreeRegressor(random_state=0)

##### Decission Tree Regression-Modell auswerten

In [273]:
score = regressor.score(x_train, y_train)
print("R-squared:", score) 

R-squared: 0.9514093034141647


In [274]:
y_pred = regressor.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)
print("RMSE: ", mse**(1/2.0)) 

MSE:  7723.960292022792
RMSE:  87.88606426517683


In [275]:
y_pred = regressor.predict([[10439, 1, 0, 1, 5, 4, 5]])
  
# print the predicted price
print("Predicted square feet: % d\n"% y_pred) 

Predicted square feet:  1333



Quelle: https://www.geeksforgeeks.org/python-decision-tree-regression-using-sklearn/
        https://www.datatechnotes.com/2020/10/regression-example-with-decisiontreeregressor.html

Im Schnitt muss mit einer Abweichung von 87.89 Quadratfuß gerechnet werden, was ca. 8.16 Quadratmetern entspricht

##### Größe der Wohnungen in Quadratfuß berechnen

In [276]:
pd.options.mode.chained_assignment = None

df_berlin_airbnb_ml = df_berlin_airbnb

df_berlin_airbnb_ml.fillna(0, inplace=True)
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Accomodates"] == '*', "Accomodates"] = 0
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Postal Code"].isnull(), "Postal Code"] = 0
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Postal Code"] == '*', "Postal Code"] = 0
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Bathrooms"] == '*', "Bathrooms"] = 0
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Bedrooms"] == '*', "Bedrooms"] = 0
df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Beds"] == '*', "Beds"] = 0


In [277]:

room_type_map = {}
df_room_type_all = df_berlin_airbnb_ml["Room Type"].unique()
for index, room_type in enumerate(df_room_type_all):
    room_type_map.update({room_type: index})

property_type_map = {}
df_property_type_all = df_berlin_airbnb_ml["Property Type"].unique()
for index, property_type in enumerate(df_property_type_all):
    property_type_map.update({property_type: index})


In [278]:

for index in range(len(df_berlin_airbnb_ml)):
    if df_berlin_airbnb_ml["Square Feet"][index] == np.nan:
        df_berlin_airbnb_ml["Square Feet"][index] = 0
    if df_berlin_airbnb_ml["Square Feet"][index] == 0:
        df_berlin_airbnb_ml["Square Feet"][index] = regressor.predict([[df_berlin_airbnb_ml["Postal Code"][index], property_type_map[df_berlin_airbnb_ml["Property Type"][index]], room_type_map[df_berlin_airbnb_ml["Room Type"][index]], df_berlin_airbnb_ml["Accomodates"][index], df_berlin_airbnb_ml["Bathrooms"][index], df_berlin_airbnb_ml["Bedrooms"][index], df_berlin_airbnb_ml["Beds"][index]]])
        df_berlin_airbnb_ml["Square Feet"][index] = round(df_berlin_airbnb_ml["Square Feet"][index], 2)

# df_berlin_airbnb_ml.loc[df_berlin_airbnb_ml["Square Feet"] > 10000, "Square Feet"] = df_berlin_airbnb_ml["Square Feet"].mean()


In [279]:

df_berlin_airbnb["Square Feet"] = df_berlin_airbnb_ml["Square Feet"]

In [280]:
df_berlin_airbnb["Square Meter"] = round(df_berlin_airbnb["Square Feet"] * 0.092903, 2)

In [281]:
df_berlin_airbnb.to_csv("../data/df_berlin_airbnb_final.csv", sep=";", decimal=",")

In [282]:
df_berlin_airbnb.head()

,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,...,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price,Square Meter
0,*,156079597.0,Maximilian,2016-01-20,0,0,f,Prenzlauer Berg,Pankow,Berlin,...,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0,23.41
1,*,5302290.0,Dulie,2013-04-07,0,0,f,Pankow,Pankow,Berlin,...,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0,74.79
2,*,59151456.0,Geank,2016-02-07,0,0,f,Prenzlauer Berg,Pankow,Berlin,...,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0,48.12
3,*,193452785.0,Alix,2018-06-26,0,0,f,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0,40.88
4,*,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,Berlin,...,0.0,0.0,0.0,0.0,0.0,0.0,t,f,55.0,32.14


In [283]:
df_berlin_airbnb.to_sql(con=db_connection, name='airbnbs', if_exists='replace')

23534

### Aggregation

#### d_zeit

In [284]:
sql_query_string = """
SELECT DISTINCT Year As Jahr
FROM crimes
"""

df_d_zeit = pd.read_sql(sql_query_string, con=db_connection)

df_d_zeit['Zeit_ID'] = np.arange(len(df_d_zeit)) + 1

df_d_zeit = df_d_zeit.reset_index().set_index("Zeit_ID", inplace=False)

df_d_zeit = df_d_zeit.drop('index', axis=1)

In [285]:
df_d_zeit.head()

,Jahr
Zeit_ID,
1,2012
2,2013
3,2014
4,2015
5,2016


#### d_subbezirk

In [286]:
sql_query_string = """
Select subbezirke.Subbezirk,
		rents.Preis_Pro_m2
from (
	select ortsteil as Subbezirk
	from rents
	union
	select Location as Subbezirk
	from crimes
	union
	select neighbourhood as Subbezirk
	from airbnbs
) as subbezirke
left join rents on rents.Ortsteil = subbezirke.Subbezirk
"""

df_d_subbezirke = pd.read_sql(sql_query_string, con=db_connection)

df_d_subbezirke['Subbezirk_ID'] = np.arange(len(df_d_subbezirke)) + 1

df_d_subbezirke = df_d_subbezirke.reset_index().set_index("Subbezirk_ID", inplace=False)

df_d_subbezirke = df_d_subbezirke.drop('index', axis=1)

In [287]:
df_d_subbezirke.head()

,Subbezirk,Preis_Pro_m2
Subbezirk_ID,,
1,Adlershof,26.84
2,Alt-Hohenschönhausen,14.01
3,Altglienicke,17.39
4,Baumschulenweg,15.37
5,Biesdorf,21.10


#### d_airbnbs

In [288]:
sql_query_string = """
select `index` as AirBnB_ID,
        `Property Type` as Immobilien_typ,
        `Room Type` as AirBnB_typ,
        Accomodates as Max_Anz_Gaeste,
        Bathrooms as Anz_Badezimmer,
        Bedrooms as Anz_Schlafzimmer,
        Beds as Anz_Betten,
        `Square Meter` as quadrat_meter
from airbnbs
"""

df_d_airbnbs = pd.read_sql(sql_query_string, con=db_connection)

df_d_airbnbs = df_d_airbnbs.reset_index().set_index("AirBnB_ID", inplace=False)

df_d_airbnbs = df_d_airbnbs.drop('index', axis=1)

In [289]:
df_d_airbnbs.head()

,Immobilien_typ,AirBnB_typ,Max_Anz_Gaeste,Anz_Badezimmer,Anz_Schlafzimmer,Anz_Betten,quadrat_meter
AirBnB_ID,,,,,,,
0,Apartment,Private room,2,1.0,1.0,1.0,23.41
1,Apartment,Entire home/apt,2,1.0,2.0,2.0,74.79
2,Apartment,Entire home/apt,3,1.0,1.0,2.0,48.12
3,Apartment,Private room,2,1.0,1.0,1.0,40.88
4,Apartment,Private room,3,1.0,1.0,2.0,32.14


#### d_sights

In [290]:
sql_query_string = """
select ID as Sight_ID,
        `Besucher pro Jahr` as Besucher_pro_Jahr,
        Sehenswürdigkeiten as Name
from sights
"""

df_d_sights = pd.read_sql(sql_query_string, con=db_connection)

df_d_sights = df_d_sights.reset_index().set_index("Sight_ID", inplace=False)

df_d_sights = df_d_sights.drop('index', axis=1)

In [291]:
df_d_sights.head()

,Besucher_pro_Jahr,Name
Sight_ID,,
1,2000000,Reichstagsgebäude
2,3000000,Brandenburger Tor
3,480000,Holocaust-Mahnmal
4,1000000,Topographie des Terrors
5,750000,Gedenkstätte Berliner Mauer


#### d_stops

In [292]:
sql_query_string = """
select ID as Stop_ID,
        stop_name as Name,
        stop_desc as Beschreibung,
        wheelchair_boarding as Behindertengerecht
from stops
"""

df_d_stops = pd.read_sql(sql_query_string, con=db_connection)

df_d_stops = df_d_stops.reset_index().set_index("Stop_ID", inplace=False)

df_d_stops = df_d_stops.drop('index', axis=1)

In [293]:
df_d_stops.head()

,Name,Beschreibung,Behindertengerecht
Stop_ID,,,
1,S Oranienburger Str. (Berlin),Ersatzhalt TucholskystraÃe vor Oranienburger ...,0
2,S Blankenfelde (TF) Bhf,S-Bahnsteig Gleis 3,1
3,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 2,1
4,"Bad Wilsnack, Bahnhof",Bahnsteig Gleis 3,1
5,"PrÃ¶sen, Bahnhof",Bahnsteig Gleis 1,1


#### f_crimes

In [294]:
sql_query_string = """
select ID as Crime_ID,
        Location as Subbezirk,
        Year as Zeit,
        (Robbery + Street_robbery + Injury + Agg_assault + Threat + Theft + Burglary + Arson) as Anzahl_Crimes
from crimes
"""

df_f_crimes = pd.read_sql(sql_query_string, con=db_connection)

df_f_crimes = df_f_crimes.reset_index().set_index("Crime_ID", inplace=False)

df_f_crimes = df_f_crimes.drop('index', axis=1)

In [295]:
df_f_crimes.head()

,Subbezirk,Zeit,Anzahl_Crimes
Crime_ID,,,
1,Tiergarten Süd,2012,3349
2,Regierungsviertel,2012,4077
3,Alexanderplatz,2012,11967
4,Brunnenstraße Süd,2012,2509
5,Moabit West,2012,3847


In [296]:
dict_subbezirke = {}

for index, row in enumerate(df_d_subbezirke["Subbezirk"]):
    dict_subbezirke[row] = index + 1

In [297]:
df_f_crimes["Subbezirk_ID"] = 'none'

In [298]:
for index, row in enumerate(df_f_crimes["Subbezirk"]):
    df_f_crimes["Subbezirk_ID"][index + 1] = dict_subbezirke[row]

In [299]:
df_f_crimes.head()

,Subbezirk,Zeit,Anzahl_Crimes,Subbezirk_ID
Crime_ID,,,,
1,Tiergarten Süd,2012,3349,75
2,Regierungsviertel,2012,4077,76
3,Alexanderplatz,2012,11967,77
4,Brunnenstraße Süd,2012,2509,78
5,Moabit West,2012,3847,79


In [300]:
dict_zeit = {}

for index, row in enumerate(df_d_zeit["Jahr"]):
    dict_zeit[row] = index + 1

In [301]:
dict_zeit

{2012: 1, 2013: 2, 2014: 3, 2015: 4, 2016: 5, 2017: 6, 2018: 7, 2019: 8}

In [302]:
df_f_crimes["Zeit_ID"] = 'none'

In [303]:
for index, row in enumerate(df_f_crimes["Zeit"]):
    df_f_crimes["Zeit_ID"][index + 1] = dict_zeit[row]

In [304]:
df_f_crimes.head()

,Subbezirk,Zeit,Anzahl_Crimes,Subbezirk_ID,Zeit_ID
Crime_ID,,,,,
1,Tiergarten Süd,2012,3349,75,1
2,Regierungsviertel,2012,4077,76,1
3,Alexanderplatz,2012,11967,77,1
4,Brunnenstraße Süd,2012,2509,78,1
5,Moabit West,2012,3847,79,1


In [305]:
df_f_crimes = df_f_crimes.drop(columns=["Zeit", "Subbezirk"])
df_f_crimes.head()

,Anzahl_Crimes,Subbezirk_ID,Zeit_ID
Crime_ID,,,
1,3349,75,1
2,4077,76,1
3,11967,77,1
4,2509,78,1
5,3847,79,1


#### f_poi

In [306]:
sql_query_string = """
select  neighbourhood as Subbezirk_ID,
        `index` as AirBnB_ID,
        null as Sight_ID,
        null as Stop_ID,
        longitude as Laengengrad,
        latitude as Breitengrad
from airbnbs
union
select  null as Subbezirk_ID,
        null as AirBnB_ID,
        ID as Sight_ID,
        null as Stop_ID,
        Longitude as Laengengrad,
        Latitude as Breitengrad
from sights
union
select  null as Subbezirk_ID,
        null as AirBnB_ID,
        null as Sight_ID,
        ID as Stop_ID,
        stop_lon as Laengengrad,
        stop_lat as Breitengrad
from stops
"""

df_f_poi = pd.read_sql(sql_query_string, con=db_connection)

df_f_poi['POI_ID'] = np.arange(len(df_f_poi)) + 1

df_f_poi = df_f_poi.reset_index().set_index("POI_ID", inplace=False)

df_f_poi = df_f_poi.drop('index', axis=1)

In [307]:
df_f_poi.head()

,Subbezirk_ID,AirBnB_ID,Sight_ID,Stop_ID,Laengengrad,Breitengrad
POI_ID,,,,,,
1,Prenzlauer Berg,0.0,NaN,NaN,13.41792,52.54652
2,Pankow,1.0,NaN,NaN,13.42214,52.56512
3,Prenzlauer Berg,2.0,NaN,NaN,13.42521,52.54741
4,Friedrichshain,3.0,NaN,NaN,13.45144,52.50958
5,Prenzlauer Berg,4.0,NaN,NaN,13.41558,52.52995


#### Entscheidungsorientiertes Datenmodell in die Datenbank einfügen

In [308]:
connection = create_server_connection("localhost", mysql_usr, mysql_pw)

# Falls schon eine OLAP_DB existiert, diese löschen:
create_database_query = "DROP DATABASE IF EXISTS olap_airbnb_db;"
create_database(connection, create_database_query)

# Erstellen der OLAP_DB
create_database_query = "CREATE DATABASE olap_airbnb_db;"
create_database(connection, create_database_query)

MySQL Database connection successful
Database created successfully
Database created successfully


In [309]:
db_connection_str = 'mysql+pymysql://' + mysql_usr + ':' + mysql_pw + '@localhost/OLAP_DB'
db_connection = create_engine(db_connection_str)

In [310]:
df_d_zeit.to_sql(con=db_connection, name='d_zeit', if_exists='replace')
df_d_subbezirke.to_sql(con=db_connection, name='d_subbezirke', if_exists='replace')
df_d_airbnbs.to_sql(con=db_connection, name='d_airbnbs', if_exists='replace')
df_d_sights.to_sql(con=db_connection, name='d_sights', if_exists='replace')
df_d_stops.to_sql(con=db_connection, name='d_stops', if_exists='replace')

df_f_crimes.to_sql(con=db_connection, name='f_crimes', if_exists='replace')
df_f_poi.to_sql(con=db_connection, name='f_poi', if_exists='replace')

64649